# TREC

In [9]:
import pandas as pd
import numpy as np

df=pd.read_csv("/Users/zhanglikang/Desktop/SC4002_G06/datasets/TREC/train.csv")
df["text"] = df["text"].str.lower()
print(len(df))
print(df.shape)
df.head()

5452
(5452, 3)


,label-coarse,label-fine,text
0,0,0,how did serfdom develop in and then leave russ...
1,1,1,what films featured the character popeye doyle ?
2,0,0,how can i find a list of celebrities ' real na...
3,1,2,what fowl grabs the spotlight after the chines...
4,2,3,what is the full form of .com ?


In [10]:
from sklearn.model_selection import train_test_split

# Split the data to create a development set of 500 examples
train_data, dev_data = train_test_split(df, test_size=500, random_state=42)

# Display the size of the training and development sets
len(train_data), len(dev_data)

(4952, 500)

In [11]:


# Get unique coarse labels
unique_labels = train_data['label-coarse'].unique()

# Randomly select 4 classes
selected_labels = np.random.choice(unique_labels, size=4, replace=False)

# Update the labels in the train and dev sets
train_data = train_data.copy()
dev_data = dev_data.copy()

train_data['new_label'] = train_data['label-coarse'].apply(lambda x: x if x in selected_labels else 'OTHERS')
dev_data['new_label'] = dev_data['label-coarse'].apply(lambda x: x if x in selected_labels else 'OTHERS')

# Display the unique labels in the updated training set
train_data['new_label'].unique()


array([2, 'OTHERS', 5, 4, 3], dtype=object)

In [5]:
import gensim.downloader

print(list(gensim.downloader.info()['models'].keys()))

# Download the "glove-twitter-25" embeddings
glove_vectors = gensim.downloader.load('word2vec-google-news-300')

# retrieve the vector for 'computer'
# glove_vectors['computer'] 

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [14]:
train_data.head()

,label-coarse,label-fine,text,new_label,avg_vector
4943,2,34,what is mikhail gorbachev 's middle initial ?,2,"[0.11785126, -0.023147583, 0.103637695, 0.1123..."
2346,0,0,how does the tail affect the flight of a kite ?,OTHERS,"[0.035095215, 0.05532837, -0.011047363, -0.013..."
1835,5,21,what were the first three cities to have a pop...,5,"[0.04341264, -0.026000977, 0.039729726, 0.0955..."
4047,1,1,what is the movie jonathan livingstone seagull ?,OTHERS,"[0.07344055, -0.024262747, -0.03922526, 0.1077..."
5097,1,23,what is a fear of home surroundings ?,OTHERS,"[0.10568237, 0.057421874, 0.087451175, 0.05285..."


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim



# Function to get the average word vector for a sentence
def sentence_vector(sentence):
    words = sentence.split()
    # Split the sentence into words.

    vectors = [glove_vectors[word] for word in words if word in glove_vectors]
    # Get the word vector for each word in the sentence if it exists in glove_vectors.


    if len(vectors) == 0: # to avoid empty lists
        return np.zeros(300)
    # If no words from the sentence are in the word vectors, return a vector of zeros.

    return np.mean(vectors, axis=0)
# Return the average word vector for the sentence.

train_data['avg_vector'] = train_data['text'].apply(sentence_vector)
# Compute the average word vector for each sentence in the training data.

X_train = np.vstack(train_data['avg_vector'].values)
# Stack the average vectors to form the training data.

y_train = pd.get_dummies(train_data['new_label']).values
# Convert the new labels to one-hot encoded vectors.

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
# Convert the training data to a PyTorch tensor.

y_train_tensor = torch.tensor(np.argmax(y_train, axis=1), dtype=torch.int64)
# Convert the one-hot encoded labels to their corresponding class indices.


In [19]:
# Neural Network Model with a simple linear layer
class SimpleClassifier(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(SimpleClassifier, self).__init__()
        # Initialize the parent class.
        
        self.fc = nn.Linear(input_dim, output_dim)
        # Define a fully connected layer.

    def forward(self, x):
        return self.fc(x)
    # Define the forward pass to return the output of the linear layer.

input_dim = 300  # as we're using word2vec-google-news-300
# Define the input dimension based on the word vector size.

output_dim = 5  # for our 5 new classes
# Define the output dimension based on the number of new classes.

model = SimpleClassifier(input_dim, output_dim)
# Initialize the model.

criterion = nn.CrossEntropyLoss()
# Define the loss function (cross entropy).

optimizer = optim.Adam(model.parameters(), lr=0.001)
# Define the optimizer (Adam) with a learning rate.

# Training
epochs = 100
# Set the number of epochs.

for epoch in range(epochs):
    optimizer.zero_grad()
    # Zero the gradients before each epoch.

    outputs = model(X_train_tensor)
    # Get the model's predictions for the training data.

    loss = criterion(outputs, y_train_tensor)
    # Compute the loss.

    loss.backward()
    # Backpropagate the loss.

    optimizer.step()
    # Update the model's parameters.

    _, predicted = torch.max(outputs, 1)
    # Get the predicted class indices.

    accuracy = (predicted == y_train_tensor).float().mean().item()
    # Compute the accuracy.

    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}, Accuracy: {accuracy}")
    # Print the loss and accuracy for each epoch.


Epoch 1/100, Loss: 1.6021628379821777, Accuracy: 0.30210015177726746
Epoch 2/100, Loss: 1.5946863889694214, Accuracy: 0.37116315960884094
Epoch 3/100, Loss: 1.5872942209243774, Accuracy: 0.41801291704177856
Epoch 4/100, Loss: 1.5799877643585205, Accuracy: 0.44204363226890564
Epoch 5/100, Loss: 1.5727678537368774, Accuracy: 0.46062198281288147
Epoch 6/100, Loss: 1.565636157989502, Accuracy: 0.4701130986213684
Epoch 7/100, Loss: 1.55859375, Accuracy: 0.46970921754837036
Epoch 8/100, Loss: 1.5516414642333984, Accuracy: 0.4691033959388733
Epoch 9/100, Loss: 1.5447801351547241, Accuracy: 0.46789175271987915
Epoch 10/100, Loss: 1.5380102396011353, Accuracy: 0.46546849608421326
Epoch 11/100, Loss: 1.5313323736190796, Accuracy: 0.4640549421310425
Epoch 12/100, Loss: 1.5247472524642944, Accuracy: 0.46122777462005615
Epoch 13/100, Loss: 1.5182543992996216, Accuracy: 0.46102583408355713
Epoch 14/100, Loss: 1.5118541717529297, Accuracy: 0.45961228013038635
Epoch 15/100, Loss: 1.5055469274520874, A